# Tiền xử lý

In [18]:
import pandas as pd

# Đọc dữ liệu từ file csv
fall_dataset = pd.read_csv("data/data.csv")

# Xóa các hàng có giá trị bị thiếu hoặc trùng lặp
fall_dataset.drop_duplicates(inplace=True)
fall_dataset.dropna(inplace=True)

# Xóa các hàng có giá trị 0 quá 50%
fall_dataset = fall_dataset.loc[(fall_dataset == 0).mean(axis=1) <= 0.5]

In [19]:
fall_dataset.head()

,1,2,3,4,5,6,7,8,9,10,...,328,329,330,331,332,333,334,335,336,label
0,0.482556,0.301600,-0.016666,0.478341,0.489260,-0.000020,0.519976,0.788471,0.122996,0.430674,...,0.474062,0.483298,0.000002,0.516464,0.785509,0.101732,0.431066,0.781687,0.098530,0
1,0.474785,0.292848,-0.047951,0.480258,0.498806,-0.000009,0.548909,0.791485,0.136930,0.465594,...,0.482976,0.486595,0.000028,0.498083,0.799704,0.090453,0.418309,0.778476,0.100237,0
2,0.491824,0.284425,0.014305,0.482035,0.483415,0.000027,0.497900,0.798342,0.084356,0.444572,...,0.477217,0.492717,0.000066,0.497273,0.792012,0.143285,0.459336,0.794777,0.134134,0
3,0.477825,0.290468,-0.051541,0.479761,0.494881,-0.000031,0.549031,0.790939,0.080075,0.462719,...,0.486793,0.495827,-0.000026,0.500054,0.796067,0.144824,0.396622,0.784166,0.043748,0
4,0.484249,0.292227,-0.006678,0.484738,0.495959,0.000036,0.499913,0.796029,0.119156,0.462000,...,0.480667,0.495396,0.000031,0.500835,0.794080,0.122698,0.462593,0.793039,0.126805,0


In [20]:
fall_dataset["label"].value_counts()

label
0    120
1     60
Name: count, dtype: int64

In [21]:
# Tăng số lượng dữ liệu bằng cách nhân bản dữ liệu
for i in range(0, 5):
    fall_dataset = pd.concat([fall_dataset, fall_dataset], ignore_index=True)

In [22]:
fall_dataset["label"].value_counts()

label
0    3840
1    1920
Name: count, dtype: int64

In [23]:
X = fall_dataset.iloc[:, :-1]
y = fall_dataset.iloc[:, -1:]

In [24]:
from imblearn.over_sampling import SMOTE

# Áp dụng Smote để cần bằng dữ liệu bằng cách tạo thêm các mẫu ví dụ cho lớp thiểu số
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [25]:
y.value_counts()

label
0        3840
1        3840
Name: count, dtype: int64

In [26]:
from sklearn.model_selection import train_test_split

# Tách tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [27]:
# Đếm mỗi loại giá trị của mỗi lớp trong tập huấn luyện và tập kiểm tra
print("Số ví dụ trong tập train: %s" % y_train.count())
print("Số ví dụ trong tập test: %s" % y_test.count())

Số ví dụ trong tập train: label    6144
dtype: int64
Số ví dụ trong tập test: label    1536
dtype: int64


In [28]:
from sklearn.preprocessing import StandardScaler

# Chuẩn hóa các đặc trưng (tùy chọn, nhưng có thể cải thiện hiệu suất cho một số mô hình)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [29]:
X_test

array([[-1.08178422, -0.35800699, -0.68104122, ..., -0.41411624,
        -1.27164476,  0.44043567],
       [ 0.44049568, -0.22791153,  0.5245661 , ...,  0.22634588,
         0.37578646,  0.04006948],
       [ 0.47118456, -0.28846694,  2.01263512, ...,  2.32400857,
         0.66983573, -1.06978508],
       ...,
       [ 0.57193881,  2.01471887, -2.07775581, ..., -0.06664789,
         0.07199283, -0.89375301],
       [ 0.54766222, -0.04820758,  0.25472838, ..., -2.62359792,
        -4.03493254, -0.22875094],
       [ 1.11078223,  1.57539008, -0.67933368, ...,  0.68649053,
         1.22124205,  2.38551712]])

# Vẽ biểu đồ 3D

In [30]:
import plotly.graph_objects as go
import pandas as pd

data_sets = [(X_train, y_train, "Train"), (X_test, y_test, "Test")]
train_colors = ["blue", "red"]
test_colors = ["green", "orange"]
visible_dict = {"Normal": True, "Anomaly": True}
fig = go.Figure()

# Tạo biểu đồ 3D cho tập huấn luyện và tập kiểm tra
for (X, y, name_prefix), colors in zip(data_sets, [train_colors, test_colors]):
    balanced_data = pd.DataFrame(
        {"x": X[:, 0], "y": X[:, 1], "z": X[:, 2], "anomaly": y}
    ).sample(n=min(50000, len(X)), random_state=42)

    for anomaly_label, color in zip(range(2), colors):
        scatter = go.Scatter3d(
            x=balanced_data[balanced_data["anomaly"] == anomaly_label]["x"],
            y=balanced_data[balanced_data["anomaly"] == anomaly_label]["y"],
            z=balanced_data[balanced_data["anomaly"] == anomaly_label]["z"],
            mode="markers",
            marker=dict(size=4, color=color, opacity=0.1),
            name=f'{name_prefix} Data: {"Normal" if anomaly_label == 0 else "Anomaly"}',
            visible=visible_dict.get("Normal", True)
            if anomaly_label == 0
            else visible_dict.get("Anomaly", True),
        )
        fig.add_trace(scatter)

# Thêm nút để chuyển đổi giữa các lớp
buttons = [
    dict(
        label="Normal",
        method="update",
        args=[
            {
                "visible": [
                    True,
                    False,
                    True,
                    False,
                    visible_dict["Anomaly"],
                    visible_dict["Anomaly"],
                ]
            }
        ],
    ),
    dict(
        label="Anomaly",
        method="update",
        args=[
            {
                "visible": [
                    False,
                    True,
                    False,
                    True,
                    visible_dict["Normal"],
                    visible_dict["Normal"],
                ]
            }
        ],
    ),
]

# Cập nhật và hiển thị biểu đồ
fig.update_layout(updatemenus=[dict(type="buttons", showactive=False, buttons=buttons)])
fig.show()

# Thuật toán học có giám sát (Supervised Learning Algorithms)

In [31]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

supervised_learning_models = {
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
}

In [32]:
from sklearn.model_selection import GridSearchCV

# Huấn luyện mô hình
train_models = {}
for name, classifier in supervised_learning_models.items():
    model = GridSearchCV(classifier, param_grid={})
    model.fit(X_train, y_train)
    train_models[name] = model

In [33]:
# Đánh giá mô hình
for name, model in train_models.items():
    # Tạo dự đoán trên tập kiểm tra X_test
    y_pred = model.predict(X_test)

    # So sánh dự đoán với giá trị từ tập kết quả y_test
    confusion_mat = pd.crosstab(
        y_test, y_pred, rownames=["Actual"], colnames=["Predicted"]
    )

    # Lấy các giá trị TP, TN, FP, FN từ confusion matrix
    TP = confusion_mat.loc[1, 1]
    TN = confusion_mat.loc[0, 0]
    FP = confusion_mat.loc[0, 1]
    FN = confusion_mat.loc[1, 0]

    # Tính toán các chỉ số đánh giá
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = (2 * precision * recall) / (precision + recall)

    # In kết quả
    print(f"{name}:")
    print(confusion_mat)
    print(
        f"Accuracy: {accuracy*100:.2f}%, Precision: {precision*100:.2f}%, Recall: {recall*100:.2f}%, F1: {f1*100:.2f}%"
    )
    print(f"Time: {model.cv_results_['mean_fit_time'][0]:.2f} (s)\n")

KNN:
Predicted    0    1
Actual             
0          778    0
1            0  758
Accuracy: 100.00%, Precision: 100.00%, Recall: 100.00%, F1: 100.00%
Time: 0.02 (s)

Decision Tree:
Predicted    0    1
Actual             
0          778    0
1            0  758
Accuracy: 100.00%, Precision: 100.00%, Recall: 100.00%, F1: 100.00%
Time: 0.18 (s)

Naive Bayes:
Predicted    0    1
Actual             
0          743   35
1            0  758
Accuracy: 97.72%, Precision: 95.59%, Recall: 100.00%, F1: 97.74%
Time: 0.05 (s)



In [34]:
# Lưu mô hình tốt nhất
import pickle

with open("../model/model.pkl", "wb") as file:
    pickle.dump(train_models["KNN"], file)